In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Mon Sep 20 16:20:58 PDT 2021


In [16]:
%load_ext autoreload
%autoreload 2

# Load data and figure out how to extract 'from' and 'to' states from transitions

In [2]:
data = csr.VivariumResults.cleaned_from_model_spec(3.1)
data.compute_total_person_time()
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'person_time']

In [3]:
data.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,sq_lns,age,transition
0,female,2022,transition_count,29,baseline,0.0,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,1209.0,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,0.0,covered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,3907.0,uncovered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...
20156,male,2026,transition_count,946,treatment_and_prevention,8004.0,covered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
20157,male,2026,transition_count,946,treatment_and_prevention,851.0,uncovered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
20158,male,2026,transition_count,946,treatment_and_prevention,20727.0,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...
20159,male,2026,transition_count,946,treatment_and_prevention,2313.0,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...


In [4]:
data.disease_transition_count

,sex,year,measure,input_draw,scenario,value,wasting_state,age,transition
0,female,2022,transition_count,29,baseline,1724.0,susceptible_to_child_wasting,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
1,female,2022,transition_count,29,baseline,497.0,mild_child_wasting,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
2,female,2022,transition_count,29,baseline,207.0,moderate_acute_malnutrition,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
3,female,2022,transition_count,29,baseline,61.0,severe_acute_malnutrition,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
...,...,...,...,...,...,...,...,...,...
34556,male,2026,transition_count,946,treatment_and_prevention,894.0,susceptible_to_child_wasting,2_to_4,susceptible_to_measles_to_measles
34557,male,2026,transition_count,946,treatment_and_prevention,314.0,mild_child_wasting,2_to_4,susceptible_to_measles_to_measles
34558,male,2026,transition_count,946,treatment_and_prevention,228.0,moderate_acute_malnutrition,2_to_4,susceptible_to_measles_to_measles
34559,male,2026,transition_count,946,treatment_and_prevention,167.0,severe_acute_malnutrition,2_to_4,susceptible_to_measles_to_measles


## Develop code to extract from and to states from transitions

In [11]:
states_from_transition_pattern = r"^(?P<from_state>\w+)_to_(?P<to_state>\w+)$"
states = (
    data.wasting_transition_count['transition']
    .str.replace("susceptible_to", "without") # Remove word 'to' from all states
    .str.extract(states_from_transition_pattern)
    .apply(lambda col: col.str.replace("without", "susceptible_to")) # Restore original state names
)
states

,from_state,to_state
0,mild_child_wasting,moderate_acute_malnutrition
1,mild_child_wasting,moderate_acute_malnutrition
2,mild_child_wasting,moderate_acute_malnutrition
3,mild_child_wasting,moderate_acute_malnutrition
...,...,...
20156,susceptible_to_child_wasting,mild_child_wasting
20157,susceptible_to_child_wasting,mild_child_wasting
20158,susceptible_to_child_wasting,mild_child_wasting
20159,susceptible_to_child_wasting,mild_child_wasting


In [12]:
data.wasting_transition_count.join(states)

,sex,year,measure,input_draw,scenario,value,sq_lns,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,0.0,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,1209.0,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,0.0,covered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,3907.0,uncovered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...
20156,male,2026,transition_count,946,treatment_and_prevention,8004.0,covered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20157,male,2026,transition_count,946,treatment_and_prevention,851.0,uncovered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20158,male,2026,transition_count,946,treatment_and_prevention,20727.0,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20159,male,2026,transition_count,946,treatment_and_prevention,2313.0,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


# Test new functions that implement the above state extration

In [41]:
data2 = csr.VivariumResults.cleaned_from_model_spec(3.1)
data2.compute_total_person_time()
data2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count',
 'person_time']

In [42]:
vars(data2).keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'stunting_state_person_time', 'population', 'ylls', 'ylds', 'cause_state_person_time', 'cause_transition_count', 'person_time'])

In [43]:
data2.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,sq_lns,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,0.0,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,1209.0,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,0.0,covered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,3907.0,uncovered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...
20156,male,2026,transition_count,946,treatment_and_prevention,8004.0,covered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20157,male,2026,transition_count,946,treatment_and_prevention,851.0,uncovered,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20158,male,2026,transition_count,946,treatment_and_prevention,20727.0,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
20159,male,2026,transition_count,946,treatment_and_prevention,2313.0,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


In [44]:
data2.cause_transition_count

,sex,year,measure,input_draw,scenario,value,wasting_state,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,1724.0,susceptible_to_child_wasting,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases
1,female,2022,transition_count,29,baseline,497.0,mild_child_wasting,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases
2,female,2022,transition_count,29,baseline,207.0,moderate_acute_malnutrition,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases
3,female,2022,transition_count,29,baseline,61.0,severe_acute_malnutrition,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...,diarrheal_diseases,susceptible_to_diarrheal_diseases
...,...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,transition_count,946,treatment_and_prevention,894.0,susceptible_to_child_wasting,2_to_4,susceptible_to_measles_to_measles,susceptible_to_measles,measles
34557,male,2026,transition_count,946,treatment_and_prevention,314.0,mild_child_wasting,2_to_4,susceptible_to_measles_to_measles,susceptible_to_measles,measles
34558,male,2026,transition_count,946,treatment_and_prevention,228.0,moderate_acute_malnutrition,2_to_4,susceptible_to_measles_to_measles,susceptible_to_measles,measles
34559,male,2026,transition_count,946,treatment_and_prevention,167.0,severe_acute_malnutrition,2_to_4,susceptible_to_measles_to_measles,susceptible_to_measles,measles


In [45]:
data2.cause_state_person_time

,sex,year,cause_state,measure,input_draw,scenario,value,wasting_state,age,cause
0,female,2022,diarrheal_diseases,state_person_time,29,baseline,30.043806,susceptible_to_child_wasting,early_neonatal,diarrheal_diseases
1,female,2022,diarrheal_diseases,state_person_time,29,baseline,8.569473,mild_child_wasting,early_neonatal,diarrheal_diseases
2,female,2022,diarrheal_diseases,state_person_time,29,baseline,3.694730,moderate_acute_malnutrition,early_neonatal,diarrheal_diseases
3,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.932238,severe_acute_malnutrition,early_neonatal,diarrheal_diseases
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,susceptible_to_measles,state_person_time,946,treatment_and_prevention,218189.255305,susceptible_to_child_wasting,2_to_4,measles
34557,male,2026,susceptible_to_measles,state_person_time,946,treatment_and_prevention,73442.981520,mild_child_wasting,2_to_4,measles
34558,male,2026,susceptible_to_measles,state_person_time,946,treatment_and_prevention,21714.850103,moderate_acute_malnutrition,2_to_4,measles
34559,male,2026,susceptible_to_measles,state_person_time,946,treatment_and_prevention,4754.462697,severe_acute_malnutrition,2_to_4,measles


In [46]:
data2.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,410.098563,uncovered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,late_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1218.859685,uncovered,late_neonatal
...,...,...,...,...,...,...,...,...,...
11516,male,2026,susceptible_to_child_wasting,state_person_time,946,treatment_and_prevention,63963.253936,covered,12_to_23_months
11517,male,2026,susceptible_to_child_wasting,state_person_time,946,treatment_and_prevention,7028.647502,uncovered,12_to_23_months
11518,male,2026,susceptible_to_child_wasting,state_person_time,946,treatment_and_prevention,196766.643395,covered,2_to_4
11519,male,2026,susceptible_to_child_wasting,state_person_time,946,treatment_and_prevention,21446.917180,uncovered,2_to_4


# Write code to compute transition rates

In [117]:
def get_transition_rates(data, entity, strata, numerator_broadcast=None, denominator_broadcast=None, **kwargs):
    """Compute the transition rates for the given entity (either 'wasting' or 'cause')."""
    transition_count = data[f"{entity}_transition_count"]
    state_person_time = data[f"{entity}_state_person_time"].rename(columns={f"{entity}_state": "from_state"})
    transition_rates = vop.ratio(
        transition_count,
        state_person_time,
        strata = vop.list_columns(strata, "from_state"),
        numerator_broadcast = vop.list_columns('transition', 'to_state', numerator_broadcast, default=[]),
        denominator_broadcast = denominator_broadcast,
        **kwargs
    )
    return transition_rates

get_transition_rates(data2, 'wasting', ['year', 'age', 'sex'])

,year,age,sex,from_state,input_draw,scenario,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,mild_child_wasting,29,baseline,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.267776,transition_count,state_person_time,1
1,2022,1-5_months,female,mild_child_wasting,29,baseline,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.365732,transition_count,state_person_time,1
2,2022,1-5_months,female,mild_child_wasting,29,treatment_and_prevention,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.267776,transition_count,state_person_time,1
3,2022,1-5_months,female,mild_child_wasting,29,treatment_and_prevention,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.365732,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10076,2026,late_neonatal,male,susceptible_to_child_wasting,829,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105487,transition_count,state_person_time,1
10077,2026,late_neonatal,male,susceptible_to_child_wasting,829,treatment_and_prevention,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105487,transition_count,state_person_time,1
10078,2026,late_neonatal,male,susceptible_to_child_wasting,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.100652,transition_count,state_person_time,1
10079,2026,late_neonatal,male,susceptible_to_child_wasting,946,treatment_and_prevention,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.100652,transition_count,state_person_time,1


In [67]:
10080

10080

In [70]:
get_transition_rates(data2, 'wasting', ['year', 'age', 'sex', 'sq_lns'])

,year,age,sex,sq_lns,from_state,input_draw,scenario,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,covered,mild_child_wasting,29,baseline,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,NaN,transition_count,state_person_time,1
1,2022,1-5_months,female,covered,mild_child_wasting,29,baseline,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,NaN,transition_count,state_person_time,1
2,2022,1-5_months,female,covered,mild_child_wasting,29,treatment_and_prevention,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,NaN,transition_count,state_person_time,1
3,2022,1-5_months,female,covered,mild_child_wasting,29,treatment_and_prevention,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,NaN,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20156,2026,late_neonatal,male,uncovered,susceptible_to_child_wasting,829,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105487,transition_count,state_person_time,1
20157,2026,late_neonatal,male,uncovered,susceptible_to_child_wasting,829,treatment_and_prevention,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105487,transition_count,state_person_time,1
20158,2026,late_neonatal,male,uncovered,susceptible_to_child_wasting,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.100652,transition_count,state_person_time,1
20159,2026,late_neonatal,male,uncovered,susceptible_to_child_wasting,946,treatment_and_prevention,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.100652,transition_count,state_person_time,1


In [74]:
df1 = get_transition_rates(data2, 'cause', ['year', 'age', 'sex'])
df1

,year,age,sex,from_state,input_draw,scenario,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,diarrheal_diseases,29,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.616313,transition_count,state_person_time,1
1,2022,1-5_months,female,diarrheal_diseases,29,treatment_and_prevention,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,64.616313,transition_count,state_person_time,1
2,2022,1-5_months,female,diarrheal_diseases,223,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,59.494705,transition_count,state_person_time,1
3,2022,1-5_months,female,diarrheal_diseases,223,treatment_and_prevention,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,59.494705,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8636,2026,late_neonatal,male,susceptible_to_measles,829,baseline,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1
8637,2026,late_neonatal,male,susceptible_to_measles,829,treatment_and_prevention,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1
8638,2026,late_neonatal,male,susceptible_to_measles,946,baseline,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1
8639,2026,late_neonatal,male,susceptible_to_measles,946,treatment_and_prevention,susceptible_to_measles_to_measles,measles,0.000000,transition_count,state_person_time,1


In [62]:
8640

8640

In [75]:
# Cause transition rates stratified by wasting state
df2 = get_transition_rates(data2, 'cause', ['year', 'age', 'sex'], numerator_broadcast=['wasting_state'])
df2

,year,age,sex,from_state,input_draw,scenario,transition,to_state,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,diarrheal_diseases,29,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,mild_child_wasting,12.714520,transition_count,state_person_time,1
1,2022,1-5_months,female,diarrheal_diseases,29,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,moderate_acute_malnutrition,9.589880,transition_count,state_person_time,1
2,2022,1-5_months,female,diarrheal_diseases,29,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,severe_acute_malnutrition,7.172885,transition_count,state_person_time,1
3,2022,1-5_months,female,diarrheal_diseases,29,baseline,diarrheal_diseases_to_susceptible_to_diarrheal...,susceptible_to_diarrheal_diseases,susceptible_to_child_wasting,35.139029,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34556,2026,late_neonatal,male,susceptible_to_measles,946,treatment_and_prevention,susceptible_to_measles_to_measles,measles,mild_child_wasting,0.000000,transition_count,state_person_time,1
34557,2026,late_neonatal,male,susceptible_to_measles,946,treatment_and_prevention,susceptible_to_measles_to_measles,measles,moderate_acute_malnutrition,0.000000,transition_count,state_person_time,1
34558,2026,late_neonatal,male,susceptible_to_measles,946,treatment_and_prevention,susceptible_to_measles_to_measles,measles,severe_acute_malnutrition,0.000000,transition_count,state_person_time,1
34559,2026,late_neonatal,male,susceptible_to_measles,946,treatment_and_prevention,susceptible_to_measles_to_measles,measles,susceptible_to_child_wasting,0.000000,transition_count,state_person_time,1


# Summing the wasting-state-stratified cause transition rates should give the same thing as the unstratified version

## WTF? Why is this assertion failing when using `.compare` says the values only differ by `1e-14`?

Ok, something weird must be going on if it's still failing with `rtol=10`...

In [89]:
def assert_values_equal(df1, df2, **kwargs):
    df1 = vop.value(df1)
    df2 = vop.value(df2).reindex(df1.index)
    pd.testing.assert_frame_equal(df1, df2, **kwargs)

def compare_values(df1, df2, **kwargs):
    df1 = vop.value(df1)
    df2 = vop.value(df2).reindex(df1.index)
    return df1.compare(df2, **kwargs)

In [107]:
assert_values_equal(df1, vop.marginalize(df2, 'wasting_state'), rtol=10, atol=10)

AssertionError: DataFrame.iloc[:, 0] (column name="value") are different

DataFrame.iloc[:, 0] (column name="value") values are different (5.55556 %)
[index]: [(1-5_months, state_person_time, diarrheal_diseases, 29, 1, transition_count, baseline, female, susceptible_to_diarrheal_diseases, diarrheal_diseases_to_susceptible_to_diarrheal_diseases, 2022), (1-5_months, state_person_time, diarrheal_diseases, 29, 1, transition_count, treatment_and_prevention, female, susceptible_to_diarrheal_diseases, diarrheal_diseases_to_susceptible_to_diarrheal_diseases, 2022), (1-5_months, state_person_time, diarrheal_diseases, 223, 1, transition_count, baseline, female, susceptible_to_diarrheal_diseases, diarrheal_diseases_to_susceptible_to_diarrheal_diseases, 2022), (1-5_months, state_person_time, diarrheal_diseases, 223, 1, transition_count, treatment_and_prevention, female, susceptible_to_diarrheal_diseases, diarrheal_diseases_to_susceptible_to_diarrheal_diseases, 2022), (1-5_months, state_person_time

In [84]:
vop.value(df1)

value
age           denominator_measure from_state             input_draw multiplier numerator_measure scenario                 sex    to_state                          transition                                         year           
1-5_months    state_person_time   diarrheal_diseases     29         1          transition_count  baseline                 female susceptible_to_diarrheal_diseases diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  64.616313
                                                                                                 treatment_and_prevention female susceptible_to_diarrheal_diseases diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  64.616313
                                                         223        1          transition_count  baseline                 female susceptible_to_diarrheal_diseases diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  59.494705
                                                                                                 treatment_and_prevention female susceptible_to_diarrheal_diseases diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  59.494705
...                                                                                                                                                                                                                               ...
late_neonatal state_person_time   susceptible_to_measles 829        1          transition_count  baseline                 male   measles                           susceptible_to_measles_to_measles                  2026   0.000000
                                                                                                 treatment_and_prevention male   measles                           susceptible_to_measles_to_measles                  2026   0.000000
                                                         946        1          transition_count  baseline                 male   measles                           susceptible_to_measles_to_measles                  2026   0.000000
                                                                                                 treatment_and_prevention male   measles                           susceptible_to_measles_to_measles                  2026   0.000000

[8640 rows x 1 columns]

In [86]:
vop.marginalize(df2, 'wasting_state', reset_index=False)

value
age           denominator_measure from_state             input_draw multiplier numerator_measure scenario                 sex    to_state                          transition                                         year           
1-5_months    state_person_time   diarrheal_diseases     29         1          transition_count  baseline                 female susceptible_to_diarrheal_diseases diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  64.616313
                                                                                                                                                                                                                      2023  64.836841
                                                                                                                                                                                                                      2024  65.006785
                                                                                                                                                                                                                      2025  64.826701
...                                                                                                                                                                                                                               ...
late_neonatal state_person_time   susceptible_to_measles 946        1          transition_count  treatment_and_prevention male   measles                           susceptible_to_measles_to_measles                  2023   0.000000
                                                                                                                                                                                                                      2024   0.000000
                                                                                                                                                                                                                      2025   0.000000
                                                                                                                                                                                                                      2026   0.000000

[8640 rows x 1 columns]

In [97]:
diff = compare_values(df1, vop.marginalize(df2, 'wasting_state'))
diff

value  \
                                                                                                                                                                                                                                                                self   
age           denominator_measure from_state                                  input_draw multiplier numerator_measure scenario                 sex    to_state                                    transition                                         year              
1-5_months    state_person_time   diarrheal_diseases                          680        1          transition_count  baseline                 female susceptible_to_diarrheal_diseases           diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  64.549137   
                                                                              829        1          transition_count  treatment_and_prevention female susceptible_to_diarrheal_diseases           diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  70.786373   
                                  lower_respiratory_infections                29         1          transition_count  baseline                 female susceptible_to_lower_respiratory_infections lower_respiratory_infections_to_susceptible_to_... 2022  44.326024   
                                                                              602        1          transition_count  baseline                 female susceptible_to_lower_respiratory_infections lower_respiratory_infections_to_susceptible_to_... 2022  43.033830   
...                                                                                                                                                                                                                                                              ...   
late_neonatal state_person_time   susceptible_to_lower_respiratory_infections 232        1          transition_count  treatment_and_prevention male   lower_respiratory_infections                susceptible_to_lower_respiratory_infections_to_... 2026   0.093507   
                                                                              394        1          transition_count  treatment_and_prevention male   lower_respiratory_infections                susceptible_to_lower_respiratory_infections_to_... 2026   0.079504   
                                                                              602        1          transition_count  baseline                 male   lower_respiratory_infections                susceptible_to_lower_respiratory_infections_to_... 2026   0.068500   
                                                                              829        1          transition_count  baseline                 male   lower_respiratory_infections                susceptible_to_lower_respiratory_infections_to_... 2026   0.074582   

                                                                                                                                                                                                                                                                      
                                                                                                                                                                                                                                                               other  
age           denominator_measure from_state                                  input_draw multiplier numerator_measure scenario                 sex    to_state                                    transition                                         year             
1-5_months    state_person_time   diarrheal_diseases                          680        1          transition_count  baseline                 female susceptible_to_diarrheal_diseases           diarrheal_diseases_to_susceptible_to_diarrheal_... 2022  64.549137  
                                  

In [100]:
diff.iat[0, 0]

64.5491372279004

In [101]:
diff.iat[0, 1]

64.54913722790039

In [103]:
(diff.iloc[:,0] - diff.iloc[:,1]).max()

1.4210854715202004e-14

In [104]:
((diff.iloc[:,0] - diff.iloc[:,1]) / diff.iloc[:,1]).max()

2.2190841919549752e-16

In [108]:
diff.dtypes

value  self     float64
       other    float64
dtype: object

# Oh, maybe it's failing because of `NaN`'s...

Yep, it looks like the problem is nonexistent measles data in the neonatal age groups. Filling `df1`'s `NaN`s with 0 fixes it.

In [109]:
df1.value.isna().any()

True

In [111]:
df1.loc[df1.value.isna()]

,year,age,sex,from_state,input_draw,scenario,transition,to_state,value,numerator_measure,denominator_measure,multiplier
1200,2022,early_neonatal,female,measles,29,baseline,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
1201,2022,early_neonatal,female,measles,29,treatment_and_prevention,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
1202,2022,early_neonatal,female,measles,223,baseline,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
1203,2022,early_neonatal,female,measles,223,treatment_and_prevention,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8564,2026,late_neonatal,male,measles,829,baseline,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
8565,2026,late_neonatal,male,measles,829,treatment_and_prevention,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
8566,2026,late_neonatal,male,measles,946,baseline,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1
8567,2026,late_neonatal,male,measles,946,treatment_and_prevention,measles_to_susceptible_to_measles,susceptible_to_measles,NaN,transition_count,state_person_time,1


In [113]:
diff.isna().any()

value  self      True
       other    False
dtype: bool

In [115]:
diff.loc[diff[('value', 'self')].isna()]

value  \
                                                                                                                                                                                     self   
age            denominator_measure from_state input_draw multiplier numerator_measure scenario                 sex    to_state               transition                        year         
early_neonatal state_person_time   measles    29         1          transition_count  baseline                 female susceptible_to_measles measles_to_susceptible_to_measles 2022   NaN   
                                                                                      treatment_and_prevention female susceptible_to_measles measles_to_susceptible_to_measles 2022   NaN   
                                              223        1          transition_count  baseline                 female susceptible_to_measles measles_to_susceptible_to_measles 2022   NaN   
                                                                                      treatment_and_prevention female susceptible_to_measles measles_to_susceptible_to_measles 2022   NaN   
...                                                                                                                                                                                   ...   
late_neonatal  state_person_time   measles    829        1          transition_count  baseline                 male   susceptible_to_measles measles_to_susceptible_to_measles 2026   NaN   
                                                                                      treatment_and_prevention male   susceptible_to_measles measles_to_susceptible_to_measles 2026   NaN   
                                              946        1          transition_count  baseline                 male   susceptible_to_measles measles_to_susceptible_to_measles 2026   NaN   
                                                                                      treatment_and_prevention male   susceptible_to_measles measles_to_susceptible_to_measles 2026   NaN   

                                                                                                                                                                                           
                                                                                                                                                                                    other  
age            denominator_measure from_state input_draw multiplier numerator_measure scenario                 sex    to_state               transition                        year        
early_neonatal state_person_time   measles    29         1          transition_count  baseline                 female susceptible_to_measles measles_to_susceptible_to_measles 2022   0.0  
                                                                                      treatment_and_prevention female susceptible_to_measles measles_to_susceptible_to_measles 2022   0.0  
                                              223        1          transition_count  baseline                 female susceptible_to_measles measles_to_susceptible_to_measles 2022   0.0  
                                                                                      treatment_and_prevention female susceptible_to_measles measles_to_susceptible_to_measles 2022   0.0  
...                                                                                                                                                                                   ...  
late_neonatal  state_person_time   measles    829        1          transition_count  baseline                 male   susceptible_to_measles measles_to_susceptible_to_measles 2026   0.0  
                                                                                      treatment_and_prevention male   susceptible_to_measles measles_to_susceptible_to_measles 2026   0.0  
                               

In [116]:
assert_values_equal(df1.fillna(0), vop.marginalize(df2, 'wasting_state'))

# Test transition rate function after copying to module

In [118]:
cause_transition_rates = csr.get_transition_rates(data2, 'cause', ['year', 'age', 'sex'])
cause_transition_rates.equals(df1)

True

In [119]:
stratified_cause_transition_rates = csr.get_transition_rates(
    data2, 'cause', ['year', 'age', 'sex'], numerator_broadcast='wasting_state')
stratified_cause_transition_rates.equals(df2)

True